# 08 - MCP

## 01 - Setup

Verify that the MedRxiv vector database has been created. If not, use script 07.1 and 07.2 to download and create the database.

In [ ]:
from genscai import paths
import chromadb

DB_PATH = str(paths.output / "medrxiv.db")

client = chromadb.PersistentClient(path=DB_PATH)
client.list_collections()

Create an MCP client that uses the default genscai tools, which are defined in the genscai.tools module.

In [ ]:
from aimu.tools import MCPClient

# Required to allow nested event loops in Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

mcp_client = MCPClient()

tools = mcp_client.list_tools()
for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print(f"Input schema: {tool.inputSchema}")
    print("-" * 40)

Call a tool directly verify that it works.

In [ ]:
tool_results = mcp_client.call_tool("search_research_articles", params={"search_request": "malaria modeling"})

for result in tool_results:
    print(f"Result type: {result.type}")
    print(f"Result text:\n{result.text}")

## 02 - Chat with tools

Set up a model client for a model that will use tools.

In [ ]:
from aimu.models import OllamaClient

model_client = OllamaClient(OllamaClient.MODEL_MISTRAL_SMALL_3_1_24B)

response = model_client.chat_streamed(
    {
        "role": "system",
        "content": "You are a helpful assistant that can use tools to provide brief answers to questons about infectious diseases and disease modeling. Please briefly introduce yourself.",
    }
)

for chunk in response:
    print(chunk, end="", flush=True)

Send the model a request that should require a tool.

In [ ]:
model_client.mcp_client = mcp_client

response = model_client.chat_streamed(
    {"role": "user", "content": "current malaria modeling tools"}, tools=mcp_client.get_tools()
)

for chunk in response:
    print(chunk, end="", flush=True)

Inspect the messages in thead, to verify that there was a tool request/response.

In [ ]:
model_client.messages

## 03 - Test additional tools

Create an new MCP client that includes additional tools.

In [ ]:
mcp_config = {
    "mcpServers": {
        "genscai": {"command": "python", "args": ["../genscai/tools.py"]},
        "gitmcp": {"url": "https://gitmcp.io/InstituteforDiseaseModeling/laser"},
    }
}

mcp_client = MCPClient(mcp_config)
model_client.mcp_client = mcp_client  # Add the new MCP client to the model client

tools = mcp_client.list_tools()
for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print(f"Input schema: {tool.inputSchema}")
    print("-" * 40)

Send the model a request that requires the new tool.

In [ ]:
response = model_client.chat_streamed(
    {"role": "user", "content": "how to build a malaria model using LASER"}, tools=mcp_client.get_tools()
)

for chunk in response:
    print(chunk, end="", flush=True)

Inspect the messages.

In [ ]:
model_client.messages

Clean up. Delete the model client to ensure that the model can be removed from GPU memory.

In [ ]:
del model_client